<a href="https://colab.research.google.com/github/Chootana/RealtimeAgeGenderEstimation/blob/master/realtime_age_gender_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 解凍はドライブ外のColab上のディレクトリ(/content/)以下に展開すること
import os 
os.chdir('/content/')
dataset_name = 'megaage_asian'
zip_path = '/content/gdrive/My Drive/{}'.format(dataset_name)

if os.path.exists('/content/{}'.format(dataset_name)):
    print('dataset already exists')
else:
    !unzip  '/content/gdrive/My Drive/megaage_asian.zip'

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: __MACOSX/megaage_asian/train/._8345.jpg  
  inflating: megaage_asian/train/15531.jpg  
  inflating: __MACOSX/megaage_asian/train/._15531.jpg  
  inflating: megaage_asian/train/12538.jpg  
  inflating: __MACOSX/megaage_asian/train/._12538.jpg  
  inflating: megaage_asian/train/25394.jpg  
  inflating: __MACOSX/megaage_asian/train/._25394.jpg  
  inflating: megaage_asian/train/35187.jpg  
  inflating: __MACOSX/megaage_asian/train/._35187.jpg  
  inflating: megaage_asian/train/7710.jpg  
  inflating: __MACOSX/megaage_asian/train/._7710.jpg  
  inflating: megaage_asian/train/8423.jpg  
  inflating: __MACOSX/megaage_asian/train/._8423.jpg  
  inflating: megaage_asian/train/15257.jpg  
  inflating: __MACOSX/megaage_asian/train/._15257.jpg  
  inflating: megaage_asian/train/34299.jpg  
  inflating: __MACOSX/megaage_asian/train/._34299.jpg  
  inflating: megaage_asian/train/14149.jpg  
  inflating: __MACOSX/megaage_asian/train/._14149.jpg  
  inflat

In [0]:
# srcディレクトリへ移動
import os 
os.chdir('/content/gdrive/My Drive/RealtimeAgeGenderEstimation/src')
!ls -la

total 45
-rw------- 1 root root   643 Apr 12 14:34 config.py
-rw------- 1 root root  6148 Apr 12 14:34 .DS_Store
-rw------- 1 root root  2287 Apr 12 14:34 mega_age_asian_datasets.py
drwx------ 2 root root  4096 Apr 12 14:37 __pycache__
-rw------- 1 root root 10529 Apr 13 00:56 realtime_age_gender_estimation.ipynb
-rw------- 1 root root 13574 Apr 12 14:34 ssr_net.py
-rw------- 1 root root  7073 Apr 12 14:34 train_SSR-Net.py


In [0]:
import os

import time
import copy
import random

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

from mega_age_asian_datasets import MegaAgeAsianDatasets
from ssr_net import SSRNet
from config import Config

In [0]:
def train_model(model, dataloaders, optimizer, criterion, hyper_parameters):
    
    # [TODO] What is this?
    global lr_scheduler
    
    since = time.time()
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(hyper_parameters.num_epochs):
        print('Epoch {}/{}'.format(epoch+1, hyper_parameters.num_epochs))
        print('-' * 10)
        
        for phase in sorted(dataloaders.keys()):
            print('mode: {}'.format(phase))

            if phase == 'train':
                model.train()
            else: # eval or test
                model.eval()  # Set model to evaluate mode

            
            loss = 0.0
            corrects_3 = 0
            corrects_5 = 0
            for idx, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(hyper_parameters.device)
                labels = labels.to(hyper_parameters.device).float()
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # statistics
                loss += loss.item() * inputs.size(0)
                corrects_3 += torch.sum(torch.abs(outputs - labels) < 3)  # CA 3
                corrects_5 += torch.sum(torch.abs(outputs - labels) < 5)  # CA 5
            
            epoch_loss = loss / len(dataloaders[phase].dataset)
            CA_3 = corrects_3.double() / len(dataloaders[phase].dataset)
            CA_5 = corrects_5.double() / len(dataloaders[phase].dataset)
            
            
            print('{} Loss: {:.4f} CA_3: {:.4f}, CA_5: {:.4f}'.format(
                phase,
                epoch_loss,
                CA_3,
                CA_5
                ))

            time_elapsed = time.time() - since
            print('Complete in {:.0f}m {:.0f}s'.format(
                time_elapsed // 60,
                time_elapsed % 60
                ))
            
            # deep copy the model
            if phase == 'val' and CA_3 > best_acc:
                best_acc = CA_3
                best_model_weights = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(CA_3)
        
        lr_scheduler.step(epoch)
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60,
        time_elapsed % 60
        ))
    print('Best val CA_3: {:4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_weights)
    return model, val_acc_history

In [0]:
def load_labels(dir_base, file_name):
    path = '{}/list/{}'.format(dir_base, file_name)
    labels = open(path).readlines()
    return labels

In [0]:
args = {
    'data_base_path': '../../../../../content/megaage_asian',
    'batch_size': 50,
    'input_size': 64,
    'num_epochs': 100,
    'learning_rate': 0.0015,
    'weight_decay': 1e-4,
    'step_size': 30,
    'gamma': 0.1,
    'augmented': False,
    'load_pretrained': False
}

hyper_parameters = Config(args)

model = SSRNet(image_size=hyper_parameters.input_size)
model = model.to(hyper_parameters.device)

data_base_path = hyper_parameters.data_base_path

image_labels = load_labels(data_base_path, 'train_name.txt')
age_labels = load_labels(data_base_path, 'train_age.txt')


random.seed(2019)
random.shuffle(image_labels)
train_size = int(len(image_labels) * 0.9)
train_image_labels= image_labels[:train_size]
val_image_labels = image_labels[train_size:]


train_data_path = '{}/train'.format(data_base_path)

random.seed(2019)
random.shuffle(age_labels)
assert len(image_labels) == len(age_labels), 'size mismatch, image_labels: {}, age_labels: {}'.format(len(image_labels), len(age_labels))
train_age_labels = age_labels[:train_size]
val_age_labels = age_labels[train_size:]

train_datasets = MegaAgeAsianDatasets(
    train_image_labels, 
    train_age_labels, 
    train_data_path, 
    augment=hyper_parameters.augmented,
    )

val_datasets = MegaAgeAsianDatasets(
    val_image_labels,
    val_age_labels,
    train_data_path,
    augment=hyper_parameters.augmented,
    )


train_loader = DataLoader(
    train_datasets,
    batch_size=hyper_parameters.batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=2
    )

val_loader = DataLoader(
    val_datasets,
    batch_size=hyper_parameters.batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=2
)

test_image_labels = load_labels(data_base_path, 'test_name.txt')
test_age_labels = load_labels(data_base_path, 'test_age.txt')
test_data_path = '{}/test'.format(data_base_path)

test_datasets = MegaAgeAsianDatasets(
    test_image_labels,
    test_age_labels,
    test_data_path,
    augment=hyper_parameters.augmented,
    )

test_loader = DataLoader(
    test_datasets,
    batch_size=hyper_parameters.batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=2
    )

dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader,
}


optimizer = optim.Adam(
    model.parameters(),
    lr=hyper_parameters.learning_rate,
    weight_decay=hyper_parameters.weight_decay)

criterion = nn.L1Loss()

lr_scheduler = optim.lr_scheduler.StepLR(
    optimizer,
    step_size=hyper_parameters.step_size,
    gamma=hyper_parameters.gamma
    )

# Train and evaluate
model, hist = train_model(
    model,
    dataloaders,
    optimizer,
    criterion,
    hyper_parameters,
    )

save_model_path = '../trained_models'
if not os.path.isdir(save_model_path):
    os.makedirs(save_model_path)

torch.save(
    model.to('cpu').state_dict(), 
'{}/model_Adam_L1Loss_LRDecay_weightDecay{}_batch{}_lr{}_epoch{}_pretrained+90_64x64.pth'.format(
        save_model_path,
        hyper_parameters.weight_decay,
        hyper_parameters.batch_size,
        hyper_parameters.learning_rate,
        hyper_parameters.num_epochs
        )
)

Epoch 1/100
----------
mode: test
test Loss: 0.0788 CA_3: 0.1612, CA_5: 0.2096
Complete in 0m 5s
mode: train
train Loss: 0.0177 CA_3: 0.1356, CA_5: 0.2180
Complete in 0m 54s
mode: val
val Loss: 0.1689 CA_3: 0.1580, CA_5: 0.2687
Complete in 0m 58s
Epoch 2/100
----------
mode: test
test Loss: 0.1601 CA_3: 0.1787, CA_5: 0.3034
Complete in 1m 3s
mode: train
train Loss: 0.0186 CA_3: 0.1675, CA_5: 0.2737
Complete in 1m 51s
mode: val
val Loss: 0.1578 CA_3: 0.1575, CA_5: 0.2513
Complete in 1m 55s
Epoch 3/100
----------
mode: test
test Loss: 0.2172 CA_3: 0.1665, CA_5: 0.2608
Complete in 1m 60s
mode: train
train Loss: 0.0176 CA_3: 0.1828, CA_5: 0.2962
Complete in 2m 47s
mode: val
val Loss: 0.1260 CA_3: 0.1970, CA_5: 0.3267
Complete in 2m 51s
Epoch 4/100
----------
mode: test
test Loss: 0.1911 CA_3: 0.1970, CA_5: 0.3217
Complete in 2m 56s
mode: train
train Loss: 0.0152 CA_3: 0.1993, CA_5: 0.3250
Complete in 3m 44s
mode: val
val Loss: 0.1192 CA_3: 0.2070, CA_5: 0.3275
Complete in 3m 48s
Epoch 5/10

In [0]:
hist

[tensor(0.1580, device='cuda:0', dtype=torch.float64),
 tensor(0.1575, device='cuda:0', dtype=torch.float64),
 tensor(0.1970, device='cuda:0', dtype=torch.float64),
 tensor(0.2070, device='cuda:0', dtype=torch.float64),
 tensor(0.2323, device='cuda:0', dtype=torch.float64),
 tensor(0.2213, device='cuda:0', dtype=torch.float64),
 tensor(0.2378, device='cuda:0', dtype=torch.float64),
 tensor(0.2662, device='cuda:0', dtype=torch.float64),
 tensor(0.2505, device='cuda:0', dtype=torch.float64),
 tensor(0.2792, device='cuda:0', dtype=torch.float64),
 tensor(0.2885, device='cuda:0', dtype=torch.float64),
 tensor(0.2923, device='cuda:0', dtype=torch.float64),
 tensor(0.3050, device='cuda:0', dtype=torch.float64),
 tensor(0.3068, device='cuda:0', dtype=torch.float64),
 tensor(0.3135, device='cuda:0', dtype=torch.float64),
 tensor(0.3135, device='cuda:0', dtype=torch.float64),
 tensor(0.3247, device='cuda:0', dtype=torch.float64),
 tensor(0.3367, device='cuda:0', dtype=torch.float64),
 tensor(0.